<a href="https://www.kaggle.com/code/plumrocky/chess?scriptVersionId=123651574" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.layers import Dense
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
i = 0
pathes = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pathes.append(os.path.join(dirname, filename))
        i = i + 1

file = pathes[0]
print(file)

import shutil
shutil.copy('/kaggle/input/35-million-chess-games/all_with_filtered_anotations_since1998.txt', 'games.txt')

!pip install cbor2 chess

CLONE_URL = "https://github.com/fluistro/Capstone.git"
get_ipython().system(f"git clone {CLONE_URL}")

import sys
sys.path.append("Capstone")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/35-million-chess-games/all_with_filtered_anotations_since1998.txt
     |████████████████████████████████| 190 kB 4.4 MB/s 
     |████████████████████████████████| 149 kB 54.4 MB/s 
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
Cloning into 'Capstone'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 58 (delta 23), reused 49 (delta 14), pack-reused 0
Unpacking objects: 100% (58/58), done.


In [2]:
get_ipython().system(f"cd Capstone && git pull origin master")

From https://github.com/fluistro/Capstone
 * branch            master     -> FETCH_HEAD
Already up to date.


# process_input

In [3]:
# creates one list of positions, and one list of the moves played in those positions

import chess
import numpy as np
import json

d = open("games.txt", "r")

boards = []
#moves = []
start_squares = []
end_squares = []

def to_vec_array(fen):
    
    vecs = []
    last = False
    
    for c in fen:
        
        # B or W to move
        if last:
            
            if c == 'w':
                vecs.append(0)
            else:
                vecs.append(1)
            break
        
        # ignore
        if (c == '/'):
            continue
        
        # just before to move information
        elif (c == ' '):
            last = True
            continue
        
        # data about square
        else:
            
            vec = np.zeros(13) # empty square, K, Q, B, N, R, P, k, q, b, n, r, p
            
            # empty square
            try:
                
                x = int(c)
                
                vec[0] = 1
                for i in range(x):
                    vecs.append(vec.astype(int).tolist())
                    
            # piece
            except:
                
                if (c == 'K'):
                    vec[1] = 1
                elif (c == 'Q'):
                    vec[2] = 1
                elif (c == 'B'):
                    vec[3] = 1
                elif (c == 'N'):
                    vec[4] = 1
                elif (c == 'R'):
                    vec[5] = 1
                elif (c == 'P'):
                    vec[6] = 1
                elif (c == 'k'):
                    vec[7] = 1
                elif (c == 'q'):
                    vec[8] = 1
                elif (c == 'b'):
                    vec[9] = 1
                elif (c == 'n'):
                    vec[10] = 1
                elif (c == 'r'):
                    vec[11] = 1
                elif (c == 'p'):
                    vec[12] = 1
                
                vecs.append(vec.astype(int).tolist())
    
    #print(chars)
    
    return vecs

# skip past headers
for i in range(5):
    d.readline()

# iterate through games
for i in range(100):
    
    if i % 10 == 0:
        print(i)
    
    game = d.readline() # game
    board = chess.Board() # new board
    
    try:
        
        game = game[game.index("W1"):] # get just the moves, skip other information
    
        for m in game.split(" ")[:-1]:
            
            vec_array = to_vec_array(board.fen())
            
            move = m[m.index('.')+1:] # move in algebraic notation
            
            uci = board.parse_san(move).uci()
            
            start = uci[0:2]
            end = uci[2:4]
    
            start = chess.SQUARE_NAMES.index(start)
            end = chess.SQUARE_NAMES.index(end)
            
            start_vec = np.zeros(64)
            start_vec[start] = 1
            
            end_vec = np.zeros(64)
            end_vec[end] = 1
            
            board.push_san(move)
            
            start_squares.append(start_vec.astype(int).tolist())
            end_squares.append(end_vec.astype(int).tolist())
            boards.append(vec_array)
            
    except Exception as e:
        print(e)
        print(game)

import cbor2
        
j = {'board' : boards, 'start' : start_squares, 'end' : end_squares}
f = open("boards.json", "wb")
cbor2.dump(j, f)

print("Done")

0
10
20
30
40
50
60
70
substring not found
76 2000.06.29 1-0 2851 None 0 date_false result_false welo_false belo_true edate_false setup_false fen_false result2_false oyrange_false blen_true ### 

80
90
Done


# reinlearn

In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from keras import Sequential
from keras.layers import Dense

import json

# create X and y

with open('boards.json', 'rb') as f:
    data = cbor2.load(f)
    
x_start = []
y_start = []
x_end = []
y_end = []

for n in range(len(data['board'])):
    
    board = data['board'][n]
    board_info = []
    board_info_with_start = []
    
    for i in range(len(board) - 1):
        square = board[i]
        for s in square:
            board_info.append(s)
            board_info_with_start.append(s)
    
    board_info.append(board[-1])
    board_info_with_start.append(board[-1])
    
    x_start.append(board_info)
    
    start_square = data['start'][n]
    end_square = data['end'][n]
    
    y_start.append(start_square)
    y_end.append(end_square)
    
    for s in start_square:
        board_info_with_start.append(s)
        
    x_end.append(board_info_with_start)
        

# network 1: get starting square from board        

X_train, X_test, y_train, y_test = train_test_split(x_start, y_start, test_size=0.3)

classifier_start = Sequential()
#First Hidden Layer
classifier_start.add(Dense(512, activation='relu', kernel_initializer='random_normal', input_dim=833)) # change input_dim
#Second  Hidden Layer
classifier_start.add(Dense(256, activation='relu', kernel_initializer='random_normal'))
#Output Layer
classifier_start.add(Dense(64, activation='softmax', kernel_initializer='random_normal'))

#Compiling the neural network
classifier_start.compile(optimizer ='adam',loss='categorical_crossentropy', metrics =['accuracy'])

#Fitting the data to the training dataset
classifier_start.fit(X_train,y_train, batch_size=10, epochs=50)


# network 2: get ending square from board and starting square    

X_train, X_test2, y_train, y_test2 = train_test_split(x_end, y_end, test_size=0.3)

classifier_end = Sequential()
#First Hidden Layer
classifier_end.add(Dense(512, activation='relu', kernel_initializer='random_normal', input_dim=897)) # change input_dim
#Second  Hidden Layer
classifier_end.add(Dense(256, activation='relu', kernel_initializer='random_normal'))
#Output Layer
classifier_end.add(Dense(64, activation='softmax', kernel_initializer='random_normal'))

#Compiling the neural network
classifier_end.compile(optimizer ='adam',loss='categorical_crossentropy', metrics =['accuracy'])

#Fitting the data to the training dataset
classifier_end.fit(X_train,y_train, batch_size=10, epochs=50)


# evaluation

classifier_start.save("start_net")
classifier_end.save("end_net")

eval_model=classifier_start.evaluate(X_test, y_test)
eval_model
print(eval_model)

eval_model=classifier_end.evaluate(X_test2, y_test2)
eval_model
print(eval_model)

Epoch 1/50
484/484 [==============================] - 3s 3ms/step - loss: 3.9725 - accuracy: 0.0499
Epoch 2/50
484/484 [==============================] - 1s 2ms/step - loss: 3.2525 - accuracy: 0.1488
Epoch 3/50
484/484 [==============================] - 1s 2ms/step - loss: 2.7652 - accuracy: 0.2419
Epoch 4/50
484/484 [==============================] - 1s 3ms/step - loss: 2.3192 - accuracy: 0.3199
Epoch 5/50
484/484 [==============================] - 1s 2ms/step - loss: 1.7696 - accuracy: 0.4840
Epoch 6/50
484/484 [==============================] - 1s 2ms/step - loss: 1.4003 - accuracy: 0.5703
Epoch 7/50
484/484 [==============================] - 1s 2ms/step - loss: 1.0589 - accuracy: 0.6790
Epoch 8/50
484/484 [==============================] - 1s 2ms/step - loss: 0.7637 - accuracy: 0.7646
Epoch 9/50
484/484 [==============================] - 1s 2ms/step - loss: 0.5753 - accuracy: 0.8297
Epoch 10/50
484/484 [==============================] - 1s 2ms/step - loss: 0.4934 - accuracy: 0.8523